In [ ]:
import warnings; warnings.simplefilter('ignore')
from boto.mturk.connection import MTurkConnection
from boto.mturk.question import ExternalQuestion
from dateutil.parser import *
from boto.mturk.question import HTMLQuestion
import numpy as np
import datetime, hashlib, boto, os, pymongo, json, pandas

In [25]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [26]:
# ! ssh -fNL 27017:localhost:27017 bonnen@rxdhawkins.me

In [3]:
# conect to the mongodb
data = json.load(open('/Users/biota/snail/taskstream/tasks/credentials/auth.json'))
mongo_tunnel = 'mongodb://' + data['user'] + ':' + data['password'] + '@127.0.0.1'
connection = pymongo.MongoClient(mongo_tunnel)
data_base = connection['task_stream']
collection = data_base['match_to_sample']

In [5]:

in_progress = {} 


In [6]:

key_info = np.load('/Users/biota/snail/taskstream/tasks/credentials/snail_rootkey.npy').item()


In [7]:
context = 'sandbox'

if context == 'sandbox':
    
    host = 'mechanicalturk.sandbox.amazonaws.com'
    hit_url = 'https://workersandbox.mturk.com/mturk/preview?groupId='
    external_submit = 'https://workersandbox.mturk.com/mturk/externalSubmit'

elif context == 'live':

    print('okay... this one is for real!\n')
    host = 'mechanicalturk.amazonaws.com'
    hit_url = 'https://www.mturk.com/mturk/preview?groupId='
    external_submit = "https://www.mturk.com/mturk/externalSubmit"


In [ ]:
hit_compensation_amount = 0.01

In [8]:

# generate password from the worker id and current time
time_code = datetime.datetime.now()
unique_key = str(time_code)
password = hashlib.md5(unique_key.encode()).hexdigest()


In [9]:
question_html_format = open('question.txt', 'r').read()%(password, external_submit, len(password)+5)

In [11]:

question_contents = HTMLQuestion(question_html_format, 600)


In [12]:

mtc = MTurkConnection(aws_access_key_id= key_info['AWSAccessKeyId'], 
                      aws_secret_access_key=key_info['AWSSecretKey'], 
                      host=host)


In [13]:


response = mtc.create_hit(question=question_contents,
                          max_assignments=1,
                          title="A compensation HIT for workers with the Stanford Neuro AI lab",
                          description="All other workers will be rejected",
                          keywords="personalized HIT",
                          duration=60,
                          reward=hit_compensation_amount,
                          approval_delay=None)


In [14]:
if response.status: 
    
    for i in range(len(response)): 

        hit_type_id = response[i].HITTypeId
        hit_id = response[i].HITId   
        in_progress[hit_id] = {'bonus_amount':hit_compensation_amount}

        print('\n\npassword:', password, '\nurl:',  hit_url + hit_type_id, '\n\n')
        



password: 7bca83a06657e1446365b1843d2d3969 
url: https://workersandbox.mturk.com/mturk/preview?groupId=373MH2DFEY0QGOHSOZEF50U4BDXHB8 




In [27]:


result = mtc.get_assignments(hit_id, page_size=100)

if len(result): 

    for i_result in range(len(result)): 
        worker_id = result[i_result].WorkerId
        assignment_id = result[i_result].AssignmentId
        print(worker_id, ':', end=' ')
        try: 
            mtc.approve_assignment(assignment_id)
            print('accepted HIT')
        except: 
            print('already accepted HIT')

hit_id = list(in_progress)[0]


A33F2FVAMGJDGG : already accepted HIT


In [28]:
def extract_worker_info(worker_id, i_collection): 
    
    w_data, bonus_amount = '', ''
 
    if collection.find({'worker_id':worker_id}).count(): 
        
        has_data = 1 
        try: 
            w_data = collection.find({'worker_id':worker_id, 
                                      "data_type":"worker_feedback", 
                                      'assignment_id':assignment_id})[0]
            bonus_amount = w_data['total_bonus_earned']
        except: 
            q = collection.find({'worker_id':worker_id,'data_type':'single_trial'})
            bonus_amount = q[q.count()-1]['bonus_earned']
    else: 
        
        has_data, w_data, bonus_amount = 0, 0, 0

    return w_data, float('%.02f'%bonus_amount), has_data

In [29]:
for i_worker in range(len(result)): 
    
    worker_id = result[i_worker].WorkerId
    assignment_id = result[i_worker].AssignmentId
    w_data, bonus_amount, is_data = extract_worker_info(worker_id, 'match_to_sample')
    
    if is_data:
        
        try: 
            mtc.approve_assignment(assignment_id)
            print('%s: accepted compensation HIT, '%worker_id, end=' ')
        except: 
            print('%s: already accepted compensation HIT, '%worker_id, end=' ')
        
        
        base_pay = 0.50
        bonus_amount = bonus_amount + base_pay

        directory = 'compensation_records'

        if not os.path.exists(directory): os.makedirs(directory)

        if not os.path.exists(directory + '/' + worker_id + '_' + assignment_id + '_' + str(bonus_amount) + '.npy'):

            print('\nworking on a compensation HIT for %s'%worker_id)

            try: 
                print('n trials:', w_data['n_trials'])
                print('duration:', w_data['experiment_duration'])
                print('blocks_completed:', w_data['n_block'])
                print('total_bonus_earned:', w_data['total_bonus_earned'])
                print(w_data['worker_input'])

            except: 

                print("no feedback data for subject\n")

            bonus_amount = bonus_amount 

            MSG = input("Enter a custom message? Pressing enter reverts to default. ")
            if len(MSG) == 0: MSG = "Thanks so much for participating in our study!"
            BONUS = input("Amount to bonus worker? Pressing enter reverts to {} ".format(bonus_amount))
            if len(BONUS) == 0: BONUS = bonus_amount
            converted_bonus = boto.mturk.price.Price(BONUS) 

            s = mtc.grant_bonus(worker_id, assignment_id,  converted_bonus, MSG);
            print('success!\n')

            compensation_info = {'time':datetime.datetime.now(), 
                                 'worker_id': worker_id, 
                                 'message':MSG, 
                                 'bonus_amount':bonus_amount,
                                 'assignment_id':assignment_id, 
                                 'submission_response':{type(s): s}, 
                                 'worker_data':w_data}    

            np.save(directory + '/' + worker_id + '_' + assignment_id + '_' + str(bonus_amount), compensation_info)

        else: 

            print('already bonused for the original HIT')
    else: 
        
        print('Nothing in database for', worker_id)

A33F2FVAMGJDGG: already accepted compensation HIT,  
working on a compensation HIT for A33F2FVAMGJDGG
no feedback data for subject

Enter a custom message? Pressing enter reverts to default. great job tyler :D 
Amount to bonus worker? Pressing enter reverts to 0.54 5.42
success!



In [ ]:
def get_subject_payment_history(worker_id): 
    
    comp = [i for i in os.listdir('compensation_records/') if worker_id in i]
    compensation_hits = [np.load(os.path.join('compensation_records', i_comp)).item()  for i_comp in comp]
    recent_payments = pandas.read_csv('transaction_history_6months.csv')
    amazon_records = recent_payments[recent_payments['Recipient ID'] == worker_id]
    
    for i_sub in subs.keys(): 
        if 'subject_info' in subs[i_sub]:
            if worker_id in subs[i_sub]['subject_info'].keys(): 
                bonus_info[subs[i_sub]['hit_id']] = subs[i_sub]['subject_info'][worker_id]
    
    return amazon_records, bonus_info, compensation_hits
    

In [30]:
def compensate_worker(worker_id, assignment_id, bonus_amount):   

    try: 
        mtc.approve_assignment(assignment_id)
        print('%s: accepted compensation HIT, '%worker_id, end=' ')
    except: 
        print('%s: already accepted compensation HIT, '%worker_id, end=' ')

    base_pay = 0.50
    bonus_amount = bonus_amount + base_pay

    directory = 'compensation_records'

    if not os.path.exists(directory): os.makedirs(directory)

    if not os.path.exists(directory + '/' + worker_id + '_' + assignment_id + '_' + str(bonus_amount) + '.npy'):

        print('\nworking on a compensation HIT for %s'%worker_id)

        try: 
            print('n trials:', w_data['n_trials'])
            print('duration:', w_data['experiment_duration'])
            print('blocks_completed:', w_data['n_block'])
            print('total_bonus_earned:', w_data['total_bonus_earned'])
            print(w_data['worker_input'])

        except: 

            print("no feedback data for subject\n")

        bonus_amount = bonus_amount 

        MSG = input("Enter a custom message? Pressing enter reverts to default. ")
        if len(MSG) == 0: MSG = "Thanks so much for participating in our study!"
        BONUS = input("Amount to bonus worker? Pressing enter reverts to {} ".format(bonus_amount))
        if len(BONUS) == 0: BONUS = bonus_amount
        converted_bonus = boto.mturk.price.Price(BONUS) 

        s = mtc.grant_bonus(worker_id, assignment_id,  converted_bonus, MSG);
        print('success!\n')

        compensation_info = {'time':datetime.datetime.now(), 
                             'worker_id': worker_id, 
                             'message':MSG, 
                             'bonus_amount':bonus_amount,
                             'assignment_id':assignment_id, 
                             'submission_response':{type(s): s}, 
                             'worker_data':w_data}    

        np.save(directory + '/' + worker_id + '_' + assignment_id + '_' + str(bonus_amount), compensation_info)

    else: 

        print('already bonused for the original HIT')